In [ ]:
from clickhouse_driver import Client

In [ ]:
client = Client(
    host='localhost',
    port=9000,
    database='fleet',
    user='fleet',
    password='fleet',
    settings={'use_numpy': True},
)

In [ ]:
time_position, longitude, latitude = client.execute(
    'select time_position, longitude, latitude '
    'from clean_states '
    'order by time_position desc '
    'limit 10_000_000',
    columnar=True,
)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'time_position': time_position, 'longitude': longitude, 'latitude': latitude})

In [ ]:
df.head()

In [ ]:
import cartopy as cp

In [ ]:
projection = cp.crs.Robinson()
x_range = projection.x_limits
y_range = projection.y_limits

In [ ]:
projected = projection.transform_points(cp.crs.Geodetic(), df['longitude'], df['latitude'])
df['x'], df['y'] = projected.T[:2]
df.head()

In [ ]:
import datashader as ds
import colorcet as cc

In [ ]:
canvas = ds.Canvas(2048, 1024, x_range, y_range)
agg = canvas.points(df, 'x', 'y')
img = ds.tf.shade(agg, cmap=cc.fire)
img = img.to_pil()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('dark_background')
fig = plt.figure(figsize=(25, 25))
ax = plt.axes(projection=projection)
ax.coastlines(color='grey')
ax.add_feature(cp.feature.LAND, facecolor='#202020')
ax.add_feature(cp.feature.BORDERS, edgecolor='grey', linewidth=0.5)
ax.imshow(img, extent=(x_range + y_range), transform=projection, zorder=2)
plt.title('Traffic Density - Global', fontsize=20)
plt.show()

In [ ]:
def plot_region(projection, w, e, s, n, resx, resy, figw, figh, fontsize):
    geodetic = cp.crs.Geodetic()
    x1, y1 = projection.transform_point(w, s, geodetic)
    x2, y2 = projection.transform_point(e, n, geodetic)
    x_range = x1, x2
    y_range = y1, y2

    projected = projection.transform_points(cp.crs.Geodetic(), df['longitude'], df['latitude'])
    df['x'], df['y'] = projected.T[:2]

    canvas = ds.Canvas(resx, resy, x_range, y_range)
    agg = canvas.points(df, 'x', 'y')
    img = ds.tf.shade(agg, cmap=cc.fire)
    img = img.to_pil()

    plt.style.use('dark_background')
    fig = plt.figure(figsize=(figw, figh))
    ax = plt.axes(projection=projection)
    ax.coastlines(color='grey')
    ax.add_feature(cp.feature.LAND, facecolor='#202020')
    ax.add_feature(cp.feature.BORDERS, edgecolor='grey', linewidth=0.5)
    ax.imshow(img, extent=(x_range + y_range), transform=projection, zorder=2)
    plt.title('Traffic Density', fontsize=fontsize)
    plt.show()

In [ ]:
plot_region(cp.crs.Mercator(), -18, 50, 28, 72, 1024, 512, 15, 15, 16)

In [ ]:
plot_region(cp.crs.Mercator(), -180, -50, 0, 75, 1024, 512, 15, 15, 16)

In [ ]:
plot_region(cp.crs.Mercator(), -130, -55, 15, 55, 1024, 512, 15, 15, 12)